### Final Notebook for Prediction :

##### Importing libraries :

In [1]:
import pandas as pd
import time
import numpy as np
import zipfile
import os
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import re
from wordcloud import WordCloud
from tqdm import tqdm
#from sklearn.cross_validation import train_test_split
from datetime import datetime, date
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import coo_matrix
from scipy.sparse import hstack
from scipy.sparse import vstack
from scipy import sparse
from scipy.sparse import csr_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.metrics import make_scorer
#from sklearn.grid_search import GridSearchCV
#from sklearn.grid_search import RandomizedSearchCV
from sklearn.preprocessing import LabelBinarizer,LabelEncoder
from scipy.stats import randint as sp_randint
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import pickle
import joblib
from IPython.display import Image

##### Loading the vectorizers :

In [3]:
vect_gender = pickle.load(open(r'C:\Users\NamrataT\Desktop\CS_1\Dataset\airbnb-recruiting-new-user-bookings\final_dataset\vect_gender.pickle', 'rb'))
vect_signup_method = pickle.load(open(r'C:\Users\NamrataT\Desktop\CS_1\Dataset\airbnb-recruiting-new-user-bookings\final_dataset\vect_signup_method.pickle', 'rb'))
vect_language = pickle.load(open(r'C:\Users\NamrataT\Desktop\CS_1\Dataset\airbnb-recruiting-new-user-bookings\final_dataset\vect_language.pickle', 'rb'))

vect_affiliate_channel = pickle.load(open(r'C:\Users\NamrataT\Desktop\CS_1\Dataset\airbnb-recruiting-new-user-bookings\final_dataset\vect_affiliate_channel.pickle', 'rb'))
vect_affiliate_provider = pickle.load(open(r'C:\Users\NamrataT\Desktop\CS_1\Dataset\airbnb-recruiting-new-user-bookings\final_dataset\vect_affiliate_provider.pickle', 'rb'))
vect_first_affiliate_tracked = pickle.load(open(r'C:\Users\NamrataT\Desktop\CS_1\Dataset\airbnb-recruiting-new-user-bookings\final_dataset\vect_first_affiliate_tracked.pickle', 'rb'))

vect_signup_app = pickle.load(open(r'C:\Users\NamrataT\Desktop\CS_1\Dataset\airbnb-recruiting-new-user-bookings\final_dataset\vect_signup_app.pickle', 'rb'))
vect_first_device_type = pickle.load(open(r'C:\Users\NamrataT\Desktop\CS_1\Dataset\airbnb-recruiting-new-user-bookings\final_dataset\vect_first_device_type.pickle', 'rb'))
vect_first_browser = pickle.load(open(r'C:\Users\NamrataT\Desktop\CS_1\Dataset\airbnb-recruiting-new-user-bookings\final_dataset\vect_first_browser.pickle', 'rb'))

vect_unique_device_type = pickle.load(open(r'C:\Users\NamrataT\Desktop\CS_1\Dataset\airbnb-recruiting-new-user-bookings\final_dataset\vect_unique_device_type.pickle', 'rb'))

In [4]:
tfidf_vect_action = pickle.load(open(r'C:\Users\NamrataT\Desktop\CS_1\Dataset\airbnb-recruiting-new-user-bookings\final_dataset\tfidf_vect_action.pickle', 'rb'))
tfidf_vect_action_type = pickle.load(open(r'C:\Users\NamrataT\Desktop\CS_1\Dataset\airbnb-recruiting-new-user-bookings\final_dataset\tfidf_vect_action_type.pickle', 'rb'))
tfidf_vect_action_detail = pickle.load(open(r'C:\Users\NamrataT\Desktop\CS_1\Dataset\airbnb-recruiting-new-user-bookings\final_dataset\tfidf_vect_action_detail.pickle', 'rb'))

##### Loading the data :

In [22]:
with open(r'C:\Users\NamrataT\Desktop\CS_1\Dataset\airbnb-recruiting-new-user-bookings\final_dataset\final_column_list.txt', 'rb') as fp:   # Unpickling
    final_column_list = pickle.load(fp)
fp.close()

Y = pd.read_csv(r'C:\Users\NamrataT\Desktop\CS_1\Dataset\airbnb-recruiting-new-user-bookings\test_users.csv\raw_target.csv')
train_session_raw_df = pd.read_csv(r'C:\Users\NamrataT\Desktop\CS_1\Dataset\airbnb-recruiting-new-user-bookings\test_users.csv\train_session_raw_df.csv')

#Loading the RF Model :
clf_rf = joblib.load(r'C:\Users\NamrataT\Desktop\CS_1\Dataset\airbnb-recruiting-new-user-bookings\models\clf_rf')

In [71]:
train_df = pd.read_csv(r'C:\Users\NamrataT\Desktop\CS_1\Dataset\airbnb-recruiting-new-user-bookings\train_users_2.csv\train_users_2.csv')
session_df = pd.read_csv(r'C:\Users\NamrataT\Desktop\CS_1\Dataset\airbnb-recruiting-new-user-bookings\sessions.csv\sessions.csv')
session_df_unq_rec1 = session_df.groupby('user_id', as_index=False).agg(lambda x: x.tolist())
train_session_df = train_df.merge(session_df_unq_rec1, left_on = 'id', right_on = 'user_id', how = 'inner')

In [235]:
train_session_df = train_df.merge(session_df_unq_rec1, left_on = 'id', right_on = 'user_id', how = 'inner')

In [238]:
train_session_df.columns

Index(['id', 'date_account_created', 'timestamp_first_active',
       'date_first_booking', 'gender', 'age', 'signup_method', 'signup_flow',
       'language', 'affiliate_channel', 'affiliate_provider',
       'first_affiliate_tracked', 'signup_app', 'first_device_type',
       'first_browser', 'country_destination', 'user_id', 'action',
       'action_type', 'action_detail', 'device_type', 'secs_elapsed'],
      dtype='object')

In [100]:
train_session_df_backup = train_session_df

###### Creation of Raw data for the Final Method :

In [236]:
X_raw = train_session_df[:5]
Y_raw = Y[:5]

In [178]:
le = LabelEncoder()
Y = le.fit_transform(Y)
Y

array([7, 7, 7, ..., 7, 7, 7])

##### Defining the functions used :

###### Scoring function :

In [2]:
# https://www.kaggle.com/davidgasquez/ndcg-scorer

def dcg_score(y_true, y_score, k=5):
    
    """Discounted cumulative gain (DCG) at rank K.

    Parameters
    ----------
    y_true : array, shape = [n_samples]
        Ground truth (true relevance labels).
    y_score : array, shape = [n_samples, n_classes]
        Predicted scores.
    k : int
        Rank.

    Returns
    -------
    score : float
    """
    
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    gain = 2 ** y_true - 1

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain / discounts)


def ndcg_score(ground_truth, predictions, k=5):
    
    """Normalized discounted cumulative gain (NDCG) at rank K.

    Normalized Discounted Cumulative Gain (NDCG) measures the performance of a
    recommendation system based on the graded relevance of the recommended
    entities. It varies from 0.0 to 1.0, with 1.0 representing the ideal
    ranking of the entities.

    Parameters
    ----------
    ground_truth : array, shape = [n_samples]
        Ground truth (true labels represended as integers).
    predictions : array, shape = [n_samples, n_classes]
        Predicted probabilities.
    k : int
        Rank.

    Returns
    -------
    score : float

    Example
    -------
    >>> ground_truth = [1, 0, 2]
    >>> predictions = [[0.15, 0.55, 0.2], [0.7, 0.2, 0.1], [0.06, 0.04, 0.9]]
    >>> score = ndcg_score(ground_truth, predictions, k=2)
    1.0
    >>> predictions = [[0.9, 0.5, 0.8], [0.7, 0.2, 0.1], [0.06, 0.04, 0.9]]
    >>> score = ndcg_score(ground_truth, predictions, k=2)
    0.6666666666
    """
    
    lb = LabelBinarizer()
    lb.fit(range(predictions.shape[1] + 1))
    T = lb.transform(ground_truth)

    scores = []

    # Iterate over each y_true and compute the DCG score
    for y_true, y_score in zip(T, predictions):
        actual = dcg_score(y_true, y_score, k)
        best = dcg_score(y_true, y_true, k)
        score = float(actual) / float(best)
        scores.append(score)

    return np.mean(scores)


# NDCG Scorer function
ndcg_scorer = make_scorer(ndcg_score, needs_proba=True, k=5)

###### Age processing function :

In [11]:
#Function to replace any value of age less than 15 or greater than 2007 with the median age

def func_age_imput_median(age):
    
    """
    Function to replace any value of age less than 15 or greater than 2007 with the median age
    
    parameters: age 
    
    returns : age  
    
    """
    
    if age < 15.0 or age > 2007.0:
        return 34.0
    else:
        return age

In [12]:
def func_age_imput_year(age,year):
    
    """
    Function to check if age is between 117 and 2007. If true, it will substract that age value from the year of account 
    creation to get the exact age of the user on the year he created the account.
        
    parameters: age,account_created_year 
    
    returns : age  
    
    """
    
    res = 0
    if age > 117.0 and age <=2007.0:
        res = year-age
        return res 
    else:
        return age

##### Session data processing functions :

In [44]:
def total_sec(sec_list):
    
    """
    Function to calculate total time a particular user has spent in accessing the application.
        
    parameters: secs_elapsed 
    
    returns : total_secs_elapsed  
    
    """
    
    res = []
    
    #print(sec_list)
    
    
    #need to convert each element of the list to string so as to replace the 'nan' with '0' so that we dont get ValueError later.
    sec_list = [ str(i) for i in sec_list ] 
    sec_list = [ re.sub('nan','0',i) for i in sec_list] 
    
    #sec_list is a list of strings now. Iterating over the elements of the list.
    for i in sec_list:
        
        #print(i)
        #Converting the string to float to take the sum later
        res.append(float(i)) 
    res = sum(res)
    return res

In [14]:
from statistics import mean

def average_sec(sec_list):
    
    """
    Function to calculate average time a particular user has spent in accessing the application.
        
    parameters: secs_elapsed 
    
    returns : mean_secs_elapsed  
    
    """
    
    res = []
    
    #need to convert each element of the list to string so as to replace the 'nan' with '0' so that we dont get ValueError later.
    sec_list = [ str(i) for i in sec_list ] 
    sec_list = [ re.sub('nan','0',i) for i in sec_list] 
    
    #session_df is a list of strings now. Iterating over the elements of the list.
    for i in sec_list:
        
        #Converting the string to float to take the mean later
        res.append(float(i)) 
    res = mean(res)
    return res

In [15]:
def session_count(sec_list):
    
    """
    Function to calculate total sessions a particular user has used in accessing the application.
        
    parameters: secs_elapsed 
    
    returns : session_count  
    
    """
        
    return len(sec_list)

In [80]:
def unique_action(action):
    
    """
    Function to calculate unique actions per user.
        
    parameters: action/device_* 
    
    returns : unique_action/device_*  
    
    """
   
    action = [str(i) for i in action]
    
    action = [re.sub('nan','na',i) for i in action]
    
    action = ','.join(set(action))
    
    return action

#### PREPROCESSING THE ENTIRE RAW DATA :

In [189]:
def preprocess_data(data):
    
    """
    Function to preprocess the raw data.
    
    It performs the following tasks :
    
    a) Removing outliers
    b) Filling Missing Values
    c) Finding unique values in the sessions columns
    d) Computing sum/mean for the secs_elapsed column
    e) Dropping the columns not needed
    
    parameters: raw data
    
    returns : processed data  
    
    """
    
    #Processing 'date_account_created' :
    data['date_account_created'] = pd.to_datetime(data['date_account_created'])
    data['account_created_day'] = data['date_account_created'].dt.weekday
    data['account_created_month'] = data['date_account_created'].dt.month
    data['account_created_year'] = data['date_account_created'].dt.year
    
    print('"date_account_created" field is finished processing...')
    
    #Processing 'age' :
    data['age'] = data['age'].fillna(34.0)
    data['age'] = data['age'].apply(func_age_imput_median)
    data['age'] = data.apply(lambda x: func_age_imput_year(x['age'], x['account_created_year']), axis=1)
    
    print('"age" field is finished processing...')
    
    data['first_affiliate_tracked'] = data['first_affiliate_tracked'].fillna('untracked')
    
    print('"first_affiliate_tracked" field is finished processing...')
    
    data['Total_secs_elapsed'] = data['secs_elapsed'].apply(total_sec)
    data['Mean_secs_elapsed'] = data['secs_elapsed'].apply(average_sec)
    data['session_count'] = data['secs_elapsed'].apply(session_count)
    data['unique_action'] = data['action'].apply(unique_action)
    data['unique_action_type'] = data['action_type'].apply(unique_action)
    data['unique_action_detail'] = data['action_detail'].apply(unique_action)
    data['unique_device_type'] = data['device_type'].apply(unique_action)
    
    print('session fields are finished processing...')
    
    data = data.drop(['date_first_booking','timestamp_first_active','id','user_id','action','action_type',
                      'action_detail','device_type','secs_elapsed','date_account_created'],axis = 1, inplace = True)
    
    print('Fields already processed are dropped...')
    
    return data

#### VECTORISATION OF THE ENTIRE RAW DATA :

In [191]:
def vectorise_data(data):
    
    """
    Function to vectorise the processed data and 
    stacking the sparse columns together to create the final data matrix.
    
    parameters: processed data
    
    returns :  vectorised data  
    
    """
    
    test_session_gender_bow = vect_gender.transform(data['gender'].values)
    print('Gender column vectorisation finished...')
    test_session_signup_method_bow = vect_signup_method.transform(data['signup_method'].values)
    print('signup_method column vectorisation finished...')
    test_session_language_bow = vect_language.transform(data['language'].values)
    print('language column vectorisation finished...')
    test_session_affiliate_channel_bow = vect_affiliate_channel.transform(data['affiliate_channel'].values)
    print('affiliate_channel column vectorisation finished...')
    test_session_affiliate_provider_bow = vect_affiliate_provider.transform(data['affiliate_provider'].values)
    print('affiliate_provider column vectorisation finished...')
    test_session_first_affiliate_tracked_bow = vect_first_affiliate_tracked.transform(data['first_affiliate_tracked'].values)
    print('first_affiliate_tracked column vectorisation finished...')
    test_session_signup_app_bow = vect_signup_app.transform(data['signup_app'].values)
    print('signup_app column vectorisation finished...')
    test_session_first_device_type_bow = vect_first_device_type.transform(data['signup_app'].values)
    print('signup_app column vectorisation finished...')
    test_session_first_browser_bow = vect_first_browser.transform(data['first_browser'].values)
    print('first_browser column vectorisation finished...')
    test_session_unique_device_type_bow = vect_unique_device_type.transform(data['unique_device_type'].values)
    print('unique_device_type column vectorisation finished...')
    test_session_action_tfidf = tfidf_vect_action.transform(data['unique_action'].values)
    print('unique_action column vectorisation finished...')
    test_session_action_type_tfidf = tfidf_vect_action_type.transform(data['unique_action_type'].values)
    print('unique_action_type column vectorisation finished...')
    test_session_action_detail_tfidf = tfidf_vect_action_detail.transform(data['unique_action_detail'].values)
    print('unique_action_detail column vectorisation finished...')
    
    print('Current Columns : ')
    print(data.columns)
    data = data.drop(['gender', 'country_destination','signup_method', 'language', 'affiliate_channel','affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser','unique_action','unique_action_type','unique_action_detail','unique_device_type'],axis=1)
    print('Dropping columns that are vectorised : ')
    print(data.columns)
    print(data.shape)
    
    # Stacking the numerical columns with the vectorised columns to create the final data matrix :
    data = sparse.hstack((data,test_session_gender_bow,test_session_signup_method_bow,test_session_language_bow,test_session_affiliate_channel_bow,test_session_affiliate_provider_bow,test_session_first_affiliate_tracked_bow,test_session_signup_app_bow,test_session_first_device_type_bow,test_session_first_browser_bow,test_session_unique_device_type_bow,test_session_action_tfidf,test_session_action_type_tfidf,test_session_action_detail_tfidf)).tocsr()
    print('Vectorisation finished...')
    print('Data ready for modelling....')
    
    return data

### FIRST FINAL :

In [234]:
#https://www.kaggle.com/krutarthhd/airbnb-eda-and-xgboost

def final_method(data):
    
    """
    
    Function takes in raw data as input.
    and returns predictions for the input. 
    Here the input can be a single point or a set of points.
    
    parameters: raw data
    
    returns :  prediction for the data 
        
    """
    
    country_list = []
    start = time.process_time()
    print('START Preprocessing....\n')
    preprocess_data(data)
    print('\n START Vectorisation...\n')
    data1 = vectorise_data(data)
    
    print('\n START Modelling...\n')
    
    rf_pred = clf_rf.predict_proba(data1)
    
    
    for i in rf_pred:
        
        country_list.append(le.inverse_transform(np.argsort(i)[::-1][:5]).tolist())
        
    print('\n PREDICTIONS....\n')
    print(country_list,rf_pred)
    print('Time Taken (in secs) to return the prediction : ',time.process_time() - start)
    
    return country_list,rf_pred

In [237]:
country_list,rf_pred = final_method(X_raw)

START Preprocessing....

"date_account_created" field is finished processing...
"age" field is finished processing...
"first_affiliate_tracked" field is finished processing...
session fields are finished processing...
Fields already processed are dropped...

 START Vectorisation...

Gender column vectorisation finished...
signup_method column vectorisation finished...
language column vectorisation finished...
affiliate_channel column vectorisation finished...
affiliate_provider column vectorisation finished...
first_affiliate_tracked column vectorisation finished...
signup_app column vectorisation finished...
signup_app column vectorisation finished...
first_browser column vectorisation finished...
unique_device_type column vectorisation finished...
unique_action column vectorisation finished...
unique_action_type column vectorisation finished...
unique_action_detail column vectorisation finished...
Current Columns : 
Index(['gender', 'age', 'signup_method', 'signup_flow', 'language',


### SECOND FINAL :

In [227]:

def final_method2(raw_label,pred):
    
    """
    
    Function takes raw label and corresponding prediction as input and returns ndcg score.
    
    parameters:  ground truth , prediction
    
    return : ndcg score 
        
    """
    print('NDCG SCORE : \n')
    score = ndcg_score(raw_label,pred,5)
    print(score)
    return score

In [228]:
score = final_method2(Y_raw,rf_pred)

NDCG SCORE : 



0.7261859507142916

### SUMMARY :

In this notebook, we have loaded the train and session raw data. Since the data is present intwo different csv files, we merged the two files together. We created a subset of this merged data and passed it as raw data into the first Final_Method.

Along with this, we have loaded the vectorisers and the best model that we have saved earlier.

In that function, we have performed all the steps that we did in our preproccessing of the train data. We vectorised this proccessed dataset and created the new features. Finally we have used the best model to give the predictions. 

We have also shown the time taken to perform all these steps and have managed to keep the time very small (8 secs).

In the second Final_Method , we have passed the ground truth and the predictions and it return the NDCG Score.